In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nbdev import *
%nbdev_default_export core
%nbdev_default_class_level 3

Cells will be exported to geomechy.core,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
import numpy as np
import scipy
from scipy.sparse.linalg import spsolve
from scipy.io import loadmat
import torch
import time
from geomechy.base import BaseLinearSolver
from geomechy.utils import lanczos, arnoldi

# Solvers
> All solvers routines

## Time Integration Solvers

### Forward Euler

In [ ]:
#%nbdev_export
class ForwardEuler:
    pass

### Backward Euler

In [ ]:
#%nbdev_export
class BackwardEuler:
    pass

### Crank-Nicolson

In [ ]:
#%nbdev_export
class CrankNicolson:
    pass

### Runge Kutta -> RK4

In [ ]:
#%nbdev_export
class RK4:
    pass

## Iterative Linear Solvers

### Steepest Descent Method

In [ ]:
#%nbdev_export
class SteepestDescent(BaseLinearSolver):
    
    def __init__(self, A, b, xo, tol=1e-10, maxIter=10000):
        BaseLinearSolver.__init__(self, A, b, xo, tol, maxIter)
    
    def solve(self):
        
        r = self.b - torch.matmul(self.A,self.xo)
        x = self.xo
        iIter = 1
        
        while torch.norm(r)>=self.tol and iIter<=self.maxIter:
            alpha = (torch.matmul(r.T,r))/(torch.matmul(r.T,torch.matmul(self.A,r)))
            x = x + alpha*r
            r = self.b - torch.matmul(self.A,x)
            iIter += 1
        return x.cpu().numpy(), iIter, torch.norm(r)

In [ ]:
A  = loadmat('./assets/matlab/ACG.mat')["ACG"]#.toarray()
b  = loadmat('./assets/matlab/b.mat')["b"]
xo = loadmat('./assets/matlab/xo.mat')["xo"]
algo = SteepestDescent(A,b,xo, maxIter=2e5)

In [ ]:
start_1 = time.time()
x, iIter, error = algo.solve()
finish_1 = time.time() - start_1

print(x[1:10])
print(iIter)
print(finish_1)
print(error)

[[ 118.5629]
 [  95.7831]
 [ 275.7696]
 [ -26.7872]
 [ -25.7977]
 [  -2.9711]
 [ -66.834 ]
 [ -19.3519]
 [-276.8964]]
11
0.9877643585205078
742.4999171362787


In [ ]:
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
b_torch  = torch.from_numpy(b).type(torch.float64).to(device)
A_torch  = torch.from_numpy(A.todense()).type(torch.float64).to(device)
start_2 = time.time()
x_torch,_ = torch.solve(b_torch, A_torch)
finish_2 = time.time() - start_2
print(x_torch.cpu().numpy()[1:10])
print(finish_2)

[[ 9.0119e+01]
 [ 3.8219e+02]
 [ 1.0397e+03]
 [ 1.5116e+00]
 [-1.3236e+01]
 [ 5.6058e-01]
 [-4.0081e+02]
 [ 1.1652e+03]
 [-1.5533e+02]]
0.004999876022338867


In [ ]:
start_3 = time.time()
x_scipy = spsolve(A, b)
finish_3 = time.time() - start_3
print(x_scipy[1:10])
print(finish_3)

[ 9.0119e+01  3.8219e+02  1.0397e+03  1.5116e+00 -1.3236e+01  5.6058e-01
 -4.0081e+02  1.1652e+03 -1.5533e+02]
0.0


### Conjugate Gradient Method

In [ ]:
#%nbdev_export
class CG(BaseLinearSolver):
    def __init__(self, A, b, xo, tol=1e-10, maxIter=10000):
        BaseLinearSolver.__init__(self, A, b, xo, tol, maxIter)
    
    def solve(self):
        r = self.b - torch.matmul(self.A,self.xo)
        p = r.clone()
        x = self.xo
        tol = self.tol**2
        
        for i in range(int(self.maxIter)):
            normrsqr = torch.matmul(r.T,r)
            w = torch.matmul(self.A,p)
            alpha = normrsqr/torch.matmul(p.T,w)
            x = x + alpha*p;
            r = r - alpha*w;
            normrnewsqr = torch.matmul(r.T,r)
            if normrnewsqr<tol:
                iIter = i
                return x.cpu().numpy(), iIter, normrnewsqr.cpu().item()
            beta = normrnewsqr/normrsqr;
            p = r + beta*p;
        return x.cpu().numpy(), iIter, normrnewsqr.cpu().item()

In [ ]:
#A = loadmat('./assets/matlab/ACG.mat')["ACG"].toarray()
#b = np.random.randn(100,1)
#xo = np.random.randn(100,1)
algo = CG(A,b,xo)

In [ ]:
start_1 = time.time()
x, iIter, error = algo.solve()
finish_1 = time.time() - start_1

print(x[1:10])
print(iIter)
print(finish_1)
print(error)

[[ 9.0119e+01]
 [ 3.8219e+02]
 [ 1.0397e+03]
 [ 1.5116e+00]
 [-1.3236e+01]
 [ 5.6058e-01]
 [-4.0081e+02]
 [ 1.1652e+03]
 [-1.5533e+02]]
730
0.1699979305267334
8.034565663542085e-21


### Preconditioned Conjugate Gradient Method

In [ ]:
#%nbdev_export
class PreCG(BaseLinearSolver):
    def __init__(self, A, b, xo, tol=1e-5, maxIter=10000,method="cholesky",droptol=1.0e-4):
        BaseLinearSolver.__init__(self, A, b, xo, tol, maxIter)
        self.method  = method
        self.droptol = droptol
    
    def cholesky(self,r):
        R = torch.cholesky(self.A, upper=True)
        sol_1, _ = torch.solve(r,R.T)
        sol_2, _ = torch.solve(sol_1,R)
        return sol_2
    
    def SSOR(self, r):
        L = torch.tril(self.A, diagonal=-1)
        D = torch.diag(torch.diag(self.A))
        U = torch.triu(self.A,diagonal=1)
        sol_1, _ = torch.solve(r,L+D)
        sol_2, _ = torch.solve(torch.matmul(D,sol_1),U+D)
        
        return sol_2
    
    def solve(self):
        r = self.b - torch.matmul(self.A,self.xo)
        z = self.SSOR(r) if self.method == "SSOR" else self.cholesky(r)
        p = z.clone()
        x = self.xo.clone()
        
        for i in range(self.maxIter):
            alpha = torch.matmul(r.T,z)/torch.matmul(p.T,torch.matmul(self.A,p))
            x = x + alpha*p
            prevr = r.clone()
            prevz = z.clone()
            r = r - alpha*torch.matmul(self.A,p)
            residual = torch.norm(r)
            if residual<self.tol:
                iIter = i
                return x.cpu().numpy(), iIter, residual.cpu().item()
            z = self.SSOR(r) if self.method == "SSOR" else self.cholesky(r)
            beta = torch.matmul(r.T,z)/torch.matmul(prevr.T,prevz);
            p = z + beta*p
            iIter = i
            
        return x.cpu().numpy(), iIter, residual.cpu().item()

In [ ]:
algo = PreCG(A,b,xo)

In [ ]:
start_1 = time.time()
x, iIter, error = algo.solve()
finish_1 = time.time() - start_1

print(x[1:10])
print(iIter)
print(finish_1)
print(error)

[[ 9.0119e+01]
 [ 3.8219e+02]
 [ 1.0397e+03]
 [ 1.5116e+00]
 [-1.3236e+01]
 [ 5.6058e-01]
 [-4.0081e+02]
 [ 1.1652e+03]
 [-1.5533e+02]]
0
0.01518392562866211
2.787126526865159e-13


### MINRES Method

In [ ]:
#%nbdev_export
class MINRES(BaseLinearSolver):
    
    def __init__(self, A, b, xo, tol=1e-10, maxIter=10000, m=50):
        BaseLinearSolver.__init__(self, A, b, xo, tol, maxIter)
        self.m = m
    
    def solve(self):
        iIter=1
        while iIter <= self.maxIter:
            r        = self.b - torch.matmul(self.A,self.xo)
            Q, T     = lanczos(self.A,r,self.m,1)
            beta     = torch.norm(r)
            e1       = torch.zeros(self.m+1,1, dtype=torch.float64, device=self.device)
            e1[0]    = 1
            y,_      = torch.lstsq(beta*e1, T.to(self.device))
            x        = self.xo + Q.to(self.device)@y
            residual = torch.norm(self.b - torch.matmul(self.A,x))

            if residual < self.tol:
                return x.cpu().numpy(), iIter, residual.cpu().item()
            
            self.xo = x
            iIter = iIter + 1
        
        return x.cpu().numpy(), iIter, residual.cpu().item()

In [ ]:
algo = MINRES(A,b,xo)

In [ ]:
start_1 = time.time()
x, iIter, residual = algo.solve()
finish_1 = time.time() - start_1


print(x[1:10])
print(iIter)
print(residual)
print(finish_1)

[[ 9.0119e+01]
 [ 3.8219e+02]
 [ 1.0397e+03]
 [ 1.5116e+00]
 [-1.3236e+01]
 [ 5.6058e-01]
 [-4.0081e+02]
 [ 1.1652e+03]
 [-1.5533e+02]]
60
8.686875445481921e-11
5.862236261367798


### GMRES Method

In [ ]:
#%nbdev_export
class GMRES(BaseLinearSolver):
    def __init__(self, A, b, xo, tol=1e-10, maxIter=10000, m=100, reorthog=0):
        BaseLinearSolver.__init__(self, A, b, xo, tol, maxIter)
        self.m = m
        self.reorthog = reorthog
        
    def solve(self):
        iIter = 1
        mm = self.m
        while iIter<=self.maxIter:
            r        = self.b - torch.matmul(self.A,self.xo)
            Q, H, m  = arnoldi(self.A,r,self.m,self.reorthog)
            beta     = torch.norm(r)
            e1       = torch.zeros(m+1,1, dtype=torch.float64, device=self.device)
            e1[0]    = 1
            y,_      = torch.lstsq(beta*e1, H.to(self.device))
            x        = self.xo + Q.to(self.device)@y
            residual = torch.norm(self.b - torch.matmul(self.A,x))

            if residual < self.tol:
                return x.cpu().numpy(), iIter, residual.cpu().item()
            
            self.xo = x
            iIter = iIter + 1
            self.m = mm
        return x.cpu().numpy(), iIter, residual.cpu().item()    

In [ ]:
algo = GMRES(A,b,xo)

In [ ]:
start_1 = time.time()
x, iIter, residual = algo.solve()
finish_1 = time.time() - start_1


print(x[1:10])
print(iIter)
print(residual)
print(finish_1)

[[ 9.0119e+01]
 [ 3.8219e+02]
 [ 1.0397e+03]
 [ 1.5116e+00]
 [-1.3236e+01]
 [ 5.6058e-01]
 [-4.0081e+02]
 [ 1.1652e+03]
 [-1.5533e+02]]
1
1.9711627992040185e-13
0.7651643753051758


### Preconditioned GMRES Method

In [ ]:
#%nbdev_export
class preGMRES(BaseLinearSolver):
    def __init__(self, A, b, xo, tol=1e-10, maxIter=10000, m=100, reorthog=0):
        BaseLinearSolver.__init__(self, A, b, xo, tol, maxIter)
        self.m = m
        self.reorthog = reorthog
        
    def solve(self):
        LU, pivots = self.A.lu()
        _,L,U = torch.lu_unpack(LU, pivots)
        
        b_bar,_ = torch.solve(self.b,L)
        b_bar   = b_bar.cpu().numpy()
        A_bar,_ = torch.solve(self.A@torch.inverse(U),L)
        A_bar   = scipy.sparse.csr_matrix(A_bar.cpu().numpy())
        xo      = self.xo.cpu().numpy()
        
        algo = GMRES(A_bar,b_bar,xo, self.tol, self.maxIter, self.m, self.reorthog)
        x_bar, iIter, _ = algo.solve()
        
        x = (torch.inverse(U.cpu())@x_bar).to(self.device)
        residual = torch.norm(self.b - torch.matmul(self.A,x))
        
        return x.cpu().numpy(), iIter, residual.cpu().item()

In [ ]:
algo = preGMRES(A,b,xo)

In [ ]:
start_1 = time.time()
x, iIter, residual = algo.solve()
finish_1 = time.time() - start_1


print(x[1:10])
print(iIter)
print(residual)
print(finish_1)

[[ 9.0119e+01]
 [ 3.8219e+02]
 [ 1.0397e+03]
 [ 1.5116e+00]
 [-1.3236e+01]
 [ 5.6058e-01]
 [-4.0081e+02]
 [ 1.1652e+03]
 [-1.5533e+02]]
1
4.359860676520736e-13
0.13197016716003418


## Iterative Non Linear Solvers

### Newton-Raphson Method

In [ ]:
#%nbdev_export
class NRM:
    pass

### Modified Newton-Raphson Method

In [ ]:
#%nbdev_export
class MNRM:
    pass

In [ ]:
%nbdev_hide
notebook2script()